In [ ]:
# REGULAR PROCESSING WITH DUMB CHUNKING

import os
import glob
import json
import asyncio
import openai
import time
from usecrets import OPENAI_API_KEY
import aiofiles

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

prompt = (
    "Ты эксперт по суммаризации текста. Сформируй краткую, четкую и содержательную суммаризацию "
    "представленного текста, избегая шаблонных вводных фраз вроде 'Документ содержит' или 'В документе говорится'. "
    "Излагай все от третьего лица в нейтрально-официальном стиле. Ты должен написать не более 2-х предложений. Текст: "
)

async def get_summary_async(text, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = await asyncio.to_thread(
                openai.OpenAI().chat.completions.create,
                model="o1-mini",
                messages=[{"role": "user", "content": prompt + text}],
                max_completion_tokens=2000
            )
            summary = response.choices[0].message.content.strip()
            return summary
        except Exception as e:
            print(f"Ошибка при обработке текста: {e}. Попытка {attempt + 1} из {max_retries}.")
            await asyncio.sleep(2 ** attempt)
    return None

async def process_single_file(txt_file, summaries_dir):
    base_name = os.path.basename(txt_file)
    name, _ = os.path.splitext(base_name)
    summary_file = os.path.join(summaries_dir, f"{name}_summ.txt")
    
    if os.path.exists(summary_file):
        print(f"Файл {summary_file} уже существует. Пропускаем {txt_file}.")
        return

    print(f"Обработка файла: {txt_file}")
    try:
        async with aiofiles.open(txt_file, "r", encoding="utf-8") as f:
            text = await f.read()
    except Exception as e:
        print(f"Ошибка чтения файла {txt_file}: {e}")
        return

    if not text.strip():
        print(f"Пустой текст в файле {txt_file}. Пропускаем.")
        return

    summary = await get_summary_async(text)
    if summary:
        async with aiofiles.open(summary_file, "w", encoding="utf-8") as f:
            await f.write(summary)
        print(f"Суммаризация успешно сохранена для файла: {txt_file} -> {summary_file}")
    else:
        print(f"Не удалось получить суммаризацию для файла: {txt_file}")

async def process_file_group(file_group, summaries_dir):
    for txt_file in file_group:
        await process_single_file(txt_file, summaries_dir)

async def process_files_and_save_summaries_async(input_dir, summaries_dir, n_groups=4):
    if not os.path.exists(summaries_dir):
        os.makedirs(summaries_dir)
    
    txt_files = glob.glob(os.path.join(input_dir, "*.txt"))
    groups = [txt_files[i::n_groups] for i in range(n_groups)]
    
    tasks = [process_file_group(group, summaries_dir) for group in groups]
    await asyncio.gather(*tasks)

def generate_training_dataset_from_summaries(input_dir, summaries_dir, output_file):
    txt_files = glob.glob(os.path.join(input_dir, "*.txt"))
    training_data = []
    
    for txt_file in txt_files:
        base_name = os.path.basename(txt_file)
        name, _ = os.path.splitext(base_name)
        summary_file = os.path.join(summaries_dir, f"{name}_summ.txt")
        
        if not os.path.exists(summary_file):
            print(f"Отсутствует суммаризация для файла: {txt_file}. Пропускаем.")
            continue
        
        try:
            with open(txt_file, "r", encoding="utf-8") as f:
                text = f.read()
            with open(summary_file, "r", encoding="utf-8") as f:
                summary = f.read()
        except Exception as e:
            print(f"Ошибка чтения файла {txt_file} или {summary_file}: {e}")
            continue
        
        if not text.strip() or not summary.strip():
            print(f"Пустой текст или суммаризация для файла: {txt_file}. Пропускаем.")
            continue
        
        training_data.append({
            "text": text,
            "summary": summary
        })
        print(f"Добавлена пара для файла: {txt_file}")
    
    with open(output_file, "w", encoding="utf-8") as f:
        for entry in training_data:
            json_line = json.dumps(entry, ensure_ascii=False)
            f.write(json_line + "\n")
    
    print(f"Обучающий датасет сохранён в файл: {output_file}")

async def main():
    input_directory = "./rproc_data"
    summaries_directory = "./rproc_data_summ"
    n_groups = 4
    
    await process_files_and_save_summaries_async(input_directory, summaries_directory, n_groups)

if __name__ == "__main__":
    await main()
    
    output_jsonl = "train.jsonl"               
    generate_training_dataset_from_summaries("./rproc_data", "./rproc_data_summ", output_jsonl)


In [4]:
# CURRENT SOLUTION WITH 1024 TOKENS (ON MY TOKENIZER) CHUNKING

import os
import glob
import json
import time
import asyncio
import aiofiles
import openai

from usecrets import OPENAI_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

from transformers import AutoTokenizer


MBART_MODEL_NAME = "d0rj/ru-mbart-large-summ"
mbart_tokenizer = AutoTokenizer.from_pretrained(MBART_MODEL_NAME)

# Размер чанка ~1000 MBART-токенов (окно 1024)
MAX_MBART_LEN = 1000
# Оверлэп (кол-во токенов, которые перекрываются между соседними окнами)
OVERLAP = 100

prompt = (
    "Ты эксперт по суммаризации текста. Сформируй краткую, четкую и содержательную суммаризацию "
    "представленного текста, избегая шаблонных вводных фраз вроде 'Документ содержит' или 'В документе говорится'. "
    "Излагай всё от третьего лица в нейтрально-официальном стиле. "
    "Ты должен написать не более 2-х предложений.\n\nТекст:\n"
)

#############################################
# Функции для нарезки текста по MBART-токенам
#############################################

def split_text_into_mbart_chunks(text, tokenizer, max_len=1024, overlap=128):
    """
    Делит исходный text на чанки по max_len MBART-токенов с overlap-токенами.
    Возвращает список списков токенов (List[List[int]]).
    """
    input_ids = tokenizer.encode(text, add_special_tokens=False)
    
    chunks = []
    start = 0
    step = max_len - overlap
    while start < len(input_ids):
        end = start + max_len
        chunk_ids = input_ids[start:end]
        chunks.append(chunk_ids)
        start += step
    
    return chunks

def mbart_tokens_to_text(token_ids, tokenizer):
    return tokenizer.decode(token_ids, skip_special_tokens=True)

#############################################
# Асинхронная функция, которая вызывает API
#############################################

async def get_summary_async(text_chunk, max_retries=3):
    for attempt in range(max_retries):
        try:

            response = await asyncio.to_thread(
                openai.OpenAI().chat.completions.create,
                model="o3-mini",
                messages=[{"role": "user", "content": prompt + text_chunk}],
                max_completion_tokens=1000
            )
            summary = response.choices[0].message.content.strip()
            return summary
        except Exception as e:
            print(f"Ошибка при обработке текста (попытка {attempt+1}/{max_retries}): {e}")
            await asyncio.sleep(2 ** attempt)
    return None

#############################################
# Асинхронная обработка одного .txt-файла
# (делим на чанки, для каждого чанка вызываем API)
#############################################

async def process_single_file_async(txt_file, summaries_dir):
    base_name = os.path.basename(txt_file)
    name, _ = os.path.splitext(base_name)

    print(f"[Начало] Обработка файла: {txt_file}")

    try:
        async with aiofiles.open(txt_file, "r", encoding="utf-8") as f:
            full_text = await f.read()
    except Exception as e:
        print(f"Ошибка чтения файла {txt_file}: {e}")
        return

    if not full_text.strip():
        print(f"Пустой текст в файле {txt_file}. Пропускаем.")
        return

    mbart_chunks = split_text_into_mbart_chunks(
        full_text,
        tokenizer=mbart_tokenizer,
        max_len=MAX_MBART_LEN,
        overlap=OVERLAP
    )

    for idx, chunk_ids in enumerate(mbart_chunks):
        chunk_summary_file = os.path.join(summaries_dir, f"{name}_summ_{idx}.txt")
        
        if os.path.exists(chunk_summary_file):
            print(f"Сводка для файла {txt_file}, chunk #{idx} уже есть.")
            continue
        
        chunk_text = mbart_tokens_to_text(chunk_ids, mbart_tokenizer)
        
        summary = await get_summary_async(chunk_text)
        
        if summary:
            try:
                async with aiofiles.open(chunk_summary_file, "w", encoding="utf-8") as sf:
                    await sf.write(summary)
                print(f"Сохранена сводка для {txt_file}, chunk #{idx} -> {chunk_summary_file}")
            except Exception as e:
                print(f"Ошибка записи файла {chunk_summary_file}: {e}")
        else:
            print(f"Не удалось получить суммаризацию для {txt_file}, chunk #{idx}")

    print(f"[Завершено] Обработка файла: {txt_file}")


async def process_file_group_async(file_group, summaries_dir):
    for txt_file in file_group:
        await process_single_file_async(txt_file, summaries_dir)


async def process_files_and_save_summaries_async(input_dir, summaries_dir, n_groups=4):
    if not os.path.exists(summaries_dir):
        os.makedirs(summaries_dir)
    
    txt_files = glob.glob(os.path.join(input_dir, "*.txt"))
    if not txt_files:
        print(f"В директории {input_dir} нет .txt файлов.")
        return

    groups = [txt_files[i::n_groups] for i in range(n_groups)]
    
    tasks = [process_file_group_async(group, summaries_dir) for group in groups]
    await asyncio.gather(*tasks)


def generate_training_dataset_from_summaries(input_dir, summaries_dir, output_file):
    """
    Синхронно обходит все .txt файлы в директории input_dir.
    Для каждого файла ищет ВСЕ чанки (base_name_summ_0.txt, base_name_summ_1.txt, ...)
    Формирует пары {"text": chunk_text, "summary": chunk_summary}
    и сохраняет в JSON Lines (output_file)
    """
    txt_files = glob.glob(os.path.join(input_dir, "*.txt"))
    training_data = []
    
    for txt_file in txt_files:
        base_name = os.path.basename(txt_file)
        name, _ = os.path.splitext(base_name)
        
        try:
            with open(txt_file, "r", encoding="utf-8") as f:
                full_text = f.read()
        except Exception as e:
            print(f"Ошибка чтения файла {txt_file}: {e}")
            continue
        
        if not full_text.strip():
            print(f"Пустой текст в файле {txt_file}. Пропускаем.")
            continue
        
        mbart_chunks = split_text_into_mbart_chunks(
            full_text, mbart_tokenizer,
            max_len=MAX_MBART_LEN,
            overlap=OVERLAP
        )
        
        for idx, chunk_ids in enumerate(mbart_chunks):
            chunk_summary_file = os.path.join(summaries_dir, f"{name}_summ_{idx}.txt")
            if not os.path.exists(chunk_summary_file):
                print(f"Нет суммаризации для {txt_file}, chunk #{idx} -> {chunk_summary_file}")
                continue
            
            try:
                with open(chunk_summary_file, "r", encoding="utf-8") as sf:
                    summary = sf.read().strip()
            except Exception as e:
                print(f"Ошибка чтения файла {chunk_summary_file}: {e}")
                continue
            
            if not summary:
                print(f"Пустая суммаризация в {chunk_summary_file}. Пропускаем.")
                continue
            
            chunk_text = mbart_tokens_to_text(chunk_ids, mbart_tokenizer)
            
            training_data.append({
                "text": chunk_text,
                "summary": summary
            })
    
    if training_data:
        with open(output_file, "w", encoding="utf-8") as f:
            for entry in training_data:
                json_line = json.dumps(entry, ensure_ascii=False)
                f.write(json_line + "\n")
        print(f"Обучающий датасет сохранён в файл: {output_file}")
    else:
        print("Не создано ни одной записи в датасете. Возможно, нет чанков или суммаризаций.")


async def main():
    input_directory = "data/rproc_data"
    summaries_directory = "data/summ_smart"
    n_groups = 16
    
    await process_files_and_save_summaries_async(input_directory, summaries_directory, n_groups)
    
    output_jsonl = "train_smart.jsonl"
    generate_training_dataset_from_summaries(input_directory, summaries_directory, output_jsonl)
 
if __name__ == "__main__":
    await main()


Token indices sequence length is longer than the specified maximum sequence length for this model (1377 > 1024). Running this sequence through the model will result in indexing errors


[Начало] Обработка файла: data/rproc_data/1053.txt
[Начало] Обработка файла: data/rproc_data/1735.txt
[Начало] Обработка файла: data/rproc_data/1721.txt
[Начало] Обработка файла: data/rproc_data/1047.txt
[Начало] Обработка файла: data/rproc_data/1709.txt
[Начало] Обработка файла: data/rproc_data/289.txt
[Начало] Обработка файла: data/rproc_data/504.txt
[Начало] Обработка файла: data/rproc_data/1090.txt
[Начало] Обработка файла: data/rproc_data/262.txt
[Начало] Обработка файла: data/rproc_data/276.txt
[Начало] Обработка файла: data/rproc_data/1084.txt
[Начало] Обработка файла: data/rproc_data/510.txt
[Начало] Обработка файла: data/rproc_data/538.txt
[Начало] Обработка файла: data/rproc_data/1912.txt
[Начало] Обработка файла: data/rproc_data/1906.txt
[Начало] Обработка файла: data/rproc_data/1537.txt
Сводка для файла data/rproc_data/276.txt, chunk #0 уже есть.
Сводка для файла data/rproc_data/276.txt, chunk #1 уже есть.
[Завершено] Обработка файла: data/rproc_data/276.txt
[Начало] Обрабо